## Examples

Monster stitches the compounds together, Igor minimizes a stitched together monster in the protein,
and Victor coordinates the process. Laboratory parallelises Victor.

Walton does not do anything in the process, but it does allow the testing of conditions
(cpt. Walton is the narrator in the book).


In [ ]:
## Move the benzene!
import numpy as np
from rdkit.Chem import Draw, AllChem
from typing import *
results: List[Chem.Mol] = []
ori = Walton.from_smiles(bezene='c1ccccc1', furan='c1ccco1')
ori.ring_on_plane(mol_idx=1)  # flat on xy
ori.atom_on_axis(mol_idx=1, atom_idx=4, axis='x') # ox on axis
ori.atom_on_axis(mol_idx=1,
                 atom_idx=ori.get_centroid_of_atoms(1, 2, mol_idx=1),
                axis='x')
ori.superpose_by_map({(0,1):{0:0, 1:1, 2:2, 3:3}})
ori.show3d()
refs = [Chem.Mol(mol) for mol in ori.mols]
for i in np.arange(start=-2, stop=6, step=0.5):
    print(i)
    demo = Walton([Chem.Mol(mol) for mol in refs], superposed=True)
    #demo.translate_parallel(mol_idx=1, base_atom_idx=2, pointer_atom_idx=4, distance=i)
    demo.translate_parallel(mol_idx=1, ref_mol_idx=0,
                            base_atom_idx=2, pointer_atom_idx=4, distance=i)
    demo()
    demo.merged.SetProp('_Name', f'separation of {i:.1f}Ang')
    results.append(AllChem.RemoveHs(demo.merged))
    #demo.show3d()

subresults = refs + results #[AllChem.RemoveHs(mol) for mol in ori.mols] + [results[i] for i in [0, 5, 7, 8, 9, 11]]
Draw.MolsToGridImage(subresults,
                     legends=[mol.GetProp('_Name') for mol in subresults],
                     subImgSize=(150,150), useSVG=True,
                     molsPerRow=5)

In [ ]:
## ring collapse

from fragmenstein import Monster, Walton
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

caffeine = Chem.MolFromSmiles('CN1C=NC2=C1C(=O)N(C(=O)N2C)C')
AllChem.EmbedMolecule(caffeine)
caffeine.SetProp('_Name', 'Caffeine')

benzene = Chem.MolFromSmiles('c1ccccc1')
AllChem.EmbedMolecule(benzene)
benzene.SetProp('_Name', 'Benzene')

toluene = Chem.MolFromSmiles('c1ccccc1(C)')
AllChem.EmbedMolecule(toluene)
toluene.SetProp('_Name', 'Toluene')

demo = Walton([toluene, caffeine])
demo.ring_on_plane(mol_idx=0, ring_idx=0, plane='xy')
demo.ring_on_plane(mol_idx=1, ring_idx=0, plane='xy')
demo.superpose_by_mcs()
demo.translate_by_point(0, demo.get_centroid_of_ring(1, 1), -1)
demo()

Draw.MolsToGridImage([demo.monster.hits[0], demo.monster.hits[1], Chem.CombineMols(demo.monster.hits[0], demo.monster.hits[1]),
                      demo.monster.modifications['Collapsed hits#0'], demo.monster.modifications['Collapsed hits#1'],
                      Chem.CombineMols(demo.monster.modifications['Collapsed hits#0'], demo.monster.modifications['Collapsed hits#1']),
                     demo.merged
                     ],
                     #legends=['Toluene', 'Caffine', 'Toluene collapsed', 'Caffine collapsed'],
                     subImgSize=(150, 150),
                     useSVG=True,
                     molsPerRow=3)